In [0]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
import os
import datetime

import matplotlib.pyplot as plt
from skimage import color, exposure

%reload_ext tensorboard

from sklearn.metrics import accuracy_score
from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

In [3]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_sign


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim ==1: y_train = to_categorical(y_train)
if y_test.ndim ==1: y_test = to_categorical(y_test)

In [0]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train,params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer="Adam", metrics=['accuracy'])

  logdir =os.path.join('logs', datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      X_train,
      y_train,
      batch_size = params_fit.get('batch_size',128),
      epochs = params_fit.get('epochs',5),
      varbose = params_fit.get('verbose',1),
      validation_data = params_fit.get('validation_data', (X_train,y_train)),
      callbacks = [tensorboard_callback]
      
      )
  return model

In [0]:
def predict(model_trained, X_test, y_test, scoring=accuracy_score):
  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred =np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm,y_pred)

In [0]:
def get_cnn_v5(input_shape,num_classes):
  return Sequential([
        Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape = input_shape),
        Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
        MaxPool2D(),
        Dropout(0.3),

        Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
        Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
        MaxPool2D(),
        Dropout(0.3),

        Conv2D(filters=64, kernel_size=(3,3), activation='relu',padding='same'),
        Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
        MaxPool2D(),
        Dropout(0.3),


        Flatten(),

        Dense(1024, activation='relu'),       
        Dropout(0.3),

        Dense(1024, activation='relu'),       
        Dropout(0.3),

        Dense(num_classes, activation='softmax'),       
        ])

In [10]:
model = get_cnn_v5(input_shape,num_classes)
model_trained = train_model(model, X_train, y_train) 
predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 6s 23ms/step - loss: 2.3104 - accuracy: 0.3587 - val_loss: 0.6146 - val_accuracy: 0.8074
Epoch 2/5
272/272 [==============================] - 6s 22ms/step - loss: 0.6334 - accuracy: 0.8035 - val_loss: 0.1539 - val_accuracy: 0.9550
Epoch 3/5
272/272 [==============================] - 6s 22ms/step - loss: 0.3333 - accuracy: 0.8980 - val_loss: 0.1072 - val_accuracy: 0.9687
Epoch 4/5
272/272 [==============================] - 6s 22ms/step - loss: 0.2355 - accuracy: 0.9313 - val_loss: 0.0399 - val_accuracy: 0.9886
Epoch 5/5
272/272 [==============================] - 6s 22ms/step - loss: 0.1731 - accuracy: 0.9484 - val_loss: 0.0292 - val_accuracy: 0.9916


0.9675736961451247

In [11]:
model_trained.evaluate(X_test, y_test)

138/138 [==============================] - 0s 3ms/step - loss: 0.1852 - accuracy: 0.9676


[0.18521353602409363, 0.9675737023353577]

In [0]:
def get_model(params):
  return Sequential([
        Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape = input_shape),
        Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
        MaxPool2D(),
        Dropout(params['dropout_cnn_block_one']),

        Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
        Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
        MaxPool2D(),
        Dropout(params['dropout_cnn_block_two']),

        Conv2D(filters=128, kernel_size=(3,3), activation='relu',padding='same'),
        Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
        MaxPool2D(),
        Dropout(params['dropout_cnn_block_three']),


        Flatten(),

        Dense(1024, activation='relu'),       
        Dropout(params['dropout_dense_block_one']),

        Dense(1024, activation='relu'),       
        Dropout(params['dropout_dense_block_two']),

        Dense(num_classes, activation='softmax'),       
        ])

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer="Adam", metrics=['accuracy'])

  model.fit(
      X_train,
      y_train,
      batch_size =128, #int(params.get('batch_size',128)),
      epochs = 5,
      verbose = 0
      #validation_data = params_fit.get('validation_data', (X_train,y_train)),
      #callbacks = [tensorboard_callback]
      )
  score =model.evaluate(X_test,y_test, verbose=0)
  accuracy = score[1]
  print('accuracy={}'.format(accuracy))

  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [14]:
space = {
    'batch_size': hp.quniform('batch_size', 100,200,10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3,0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3,0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3,0.5),
    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3,0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3,0.7),

}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

accuracy=0.9594104290008545
accuracy=0.8954648375511169
accuracy=0.8405895829200745
accuracy=0.9575963616371155
accuracy=0.9480725526809692
accuracy=0.9664399027824402
accuracy=0.9548752903938293
accuracy=0.9460317492485046
accuracy=0.9419501423835754
accuracy=0.8727891445159912
accuracy=0.9081632494926453
accuracy=0.9047619104385376
accuracy=0.956916093826294
accuracy=0.9015873074531555
accuracy=0.9532879590988159
accuracy=0.9464852809906006
accuracy=0.9013605713844299
accuracy=0.8063492178916931
accuracy=0.9317460060119629
accuracy=0.8768707513809204
accuracy=0.9517006874084473
accuracy=0.9662131667137146
accuracy=0.9594104290008545
accuracy=0.9512471556663513
accuracy=0.9034013748168945
accuracy=0.9732426404953003
accuracy=0.9653061032295227
accuracy=0.9557822942733765
accuracy=0.9394558072090149
accuracy=0.9585034251213074
100%|██████████| 30/30 [10:19<00:00, 20.65s/it, best loss: -0.9732426404953003]


In [15]:
!git satus


git: 'satus' is not a git command. See 'git --help'.

The most similar command is
	status


In [16]:
!git status

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   Day2_matrix3.ipynb
	modified:   Day_3_matrix3.ipynb
	modified:   day4_matrix3.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	data/
	logs/

no changes added to commit (use "git add" and/or "git commit -a")


In [0]:
!git add

In [0]:
def train_and_predict(model):
  #model = get_cnn_v1(input_shape,num_classes)
  model_trained = train_model(model, X_train, y_train) 
  return predict(model_trained, X_test, y_test)